In [ ]:
  !nvidia-smi

Wed Jul  1 09:20:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    65W / 149W |    349MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ignore this cell


from google.colab import drive
drive.mount('/content/gdrive')
#load from drive
!mkdir  '/content/cifar' 
!cp '/content/gdrive/My Drive/cifar-10.zip' -d '/content/cifar' 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# ignore this cell

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ignore this cell

!unzip /content/cifar/cifar-10.zip -d /content/cifar/
!apt-get install p7zip-full
!rm -r -f '/content/cifar/train'
!mkdir '/content/cifar/train'
!p7zip -d -f -ao '/content/cifar/train.7z' -o '/content/cifar/train'

Archive:  /content/cifar/cifar-10.zip
  inflating: /content/cifar/sampleSubmission.csv  
  inflating: /content/cifar/test.7z  
  inflating: /content/cifar/train.7z  
  inflating: /content/cifar/trainLabels.csv  
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
/usr/bin/p7zip: ignoring unknown option -ao

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/cifar/                         1 file, 109723070 bytes (105 MiB)

Extracting archive: /c

In [ ]:
# ignore this cell

#Read label data
import pandas as pd 
id_label=pd.read_csv('/content/cifar/trainLabels.csv').values

#create a dictionary for matching id to label
id2label_dict ={}
for ele in id_label:
  id2label_dict[ele[0]] = ele[1]

In [ ]:
# ignore this cell

#load all images into memory
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
IMAGE_SIZE=32

def normalize(image:np.array):
  standard_image=image
  standard_image = cv2.resize(standard_image, (IMAGE_SIZE,IMAGE_SIZE))
  # standard_image = (standard_image-np.mean(standard_image)/np.std(standard_image))
  standard_image = standard_image/255

  return standard_image


#function return 2 lists that Input and Label
def load_image(folder):
  X = list()
  y = list()
  for i, file_name in enumerate(os.listdir(folder)):
    try:
      path_to_image = os.path.join(folder,file_name)
      image =  cv2.imread(path_to_image,1) 
      # cv2_imshow(image)
      # break
      if image is not None:
        standard_image = normalize(image)
        X.append(standard_image)
        id = int(os.path.splitext(file_name)[0])
        label=id2label_dict[id]
        y.append(label)
        if i>0 and (i+1)% 10000 == 0:
          print("[INFO] loading process: {}/{}".format(i+1, len(os.listdir(folder))))
    except Exception as e:
      print(e)
      # pass
  print("done!")
  return X,y

X_resource,y_resource = load_image(os.path.join("/content/train"))

[INFO] loading process: 10000/50000
[INFO] loading process: 20000/50000
[INFO] loading process: 30000/50000
[INFO] loading process: 40000/50000
[INFO] loading process: 50000/50000
done!


In [ ]:
# ignore this cell

#encode label
from sklearn import preprocessing
import numpy as np
le = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
y_resource_array = np.array(y_resource).reshape(-1,1)
le.fit(y_resource_array)
X_train=np.array(X_resource)
y_train = le.transform(y_resource_array)

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import torch.utils.data as td
import random,time
import matplotlib.pyplot as plt
import torchvision         

In [137]:

# load data 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, 
                                         shuffle=False, num_workers=2)



Files already downloaded and verified
Files already downloaded and verified
Training time:  1.8048056640625


In [ ]:
## check train_loader

print(len(train_loader))   # (size data / batch size = 50000 / 128 approximate.. 391)

391


In [ ]:
# ignore this cell


# create model CNN  experiments ( errors for size and shape data)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            #conv_1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            #conv_2
            nn.Conv2d(in_channels = 32, out_channels=32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.2), 
            #conv_3
            nn.Conv2d(in_channels = 32, out_channels=64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.3), 
            #conv_4
            nn.Conv2d(in_channels = 64, out_channels=128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            #conv_5
            nn.Conv2d(in_channels = 128, out_channels=128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.4), 
            
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(128*4*4, 10),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.4),
            nn.Softmax(),
)
    def forward(self, x):
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        #  fully connected
        x = self.fc_layer(x)

        return x


import numpy as np
net = CNN()
print(net)

model_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('number of parameters: ',params)

In [ ]:
# ignore this cell

import torch.nn as nn
import torch.nn.functional as F

# CNN tham khảo
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 10)
        # dropout
        self.dropout = nn.Dropout(p=.5)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flattening
        x = x.view(-1, 64 * 4 * 4)
        # fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# create a complete CNN
model = Net()
#print(model)

# move tensors to GPU if CUDA is available

In [ ]:
# final model
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture CNN tham khảo
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #conv1
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(32)
        #conv2
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)
        #conv3
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(64)
        #conv4
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(64)
        #conv5
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv5_bn = nn.BatchNorm2d(128)
        #conv6
        self.conv6 = nn.Conv2d(128, 128 ,3 ,padding = 1)
        self.conv6_bn = nn.BatchNorm2d(128)
        
        # Definition of the MaxPooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # fully connected
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 10)
        
        # defintion of dropout 
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.dropout4 = nn.Dropout(0.4)

    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = self.pool(self.conv2_bn(F.relu(self.conv2(x))))
        x = self.dropout1(x)
        x = self.conv3_bn(F.relu(self.conv3(x)))
        x = self.pool(self.conv4_bn(F.relu(self.conv4(x))))
        x = self.dropout2(x)
        x = self.conv5_bn(F.relu(self.conv5(x)))
        x = self.pool(self.conv6_bn(F.relu(self.conv6(x))))
        x = self.dropout3(x)
        
        #flatten
        x = x.view(-1, 128 * 4 * 4)
        # add dropout layer
        # 
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout4(x)
        # add 2nd hidden layer, without relu activation function
        x = F.softmax(self.fc2(x))
        return x

model = Net()
print('Done')
#print(model)
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('number of parameters: ',params)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
#model = Net().to(device)
#summary(model, (3, 32, 32))

Done
number of parameters:  551722


In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


# number of epochs : 8
# batch size 128 


In [138]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
for epoch in range(8):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 128 == 127:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 128))
            running_loss = 0.0

print('Finished Training')
end.record()
torch.cuda.synchronize()
print('Training time: ',start.elapsed_time(end)/1000) #seconds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   128] loss: 1.734
[1,   256] loss: 1.721
[1,   384] loss: 1.727
[2,   128] loss: 1.726
[2,   256] loss: 1.719
[2,   384] loss: 1.726
[3,   128] loss: 1.715
[3,   256] loss: 1.710
[3,   384] loss: 1.719
[4,   128] loss: 1.714
[4,   256] loss: 1.710
[4,   384] loss: 1.715
[5,   128] loss: 1.715
[5,   256] loss: 1.713
[5,   384] loss: 1.701
[6,   128] loss: 1.704
[6,   256] loss: 1.701
[6,   384] loss: 1.707
[7,   128] loss: 1.700
[7,   256] loss: 1.705
[7,   384] loss: 1.711
[8,   128] loss: 1.705
[8,   256] loss: 1.701
[8,   384] loss: 1.699
Finished Training
Training time:  2480.119


In [139]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 72 %
